In [11]:
import pickle

# 파일 경로
file_path = 'ysy_morphs.p'

# pickle 파일 열기
with open(file_path, 'rb') as f:
    data = pickle.load(f)

# 데이터 타입과 내용 확인
print(type(data))  # 데이터 타입 확인
print(len(data))   # 데이터의 길이 (딕셔너리일 경우 키의 개수)

# 딕셔너리인 경우 일부 데이터 출력
if isinstance(data, dict):
    # 첫 5개의 키와 값 출력
    for i, (key, value) in enumerate(data.items()):
        print(f"Key: {key}, Value: {value}")
        if i == 14:  # 5개만 출력
            break
else:
    # 리스트나 다른 타입인 경우
    print(data)  # 슬라이싱


<class 'dict'>
594
Key: 1, Value: {'paper_id': '1', 'content': ['화성', '화성시', '동탄', '센트럴파크', '국회', '기자단', '평택', '류미', '이틀', '최대', '승부처', '수도', '막판', '스퍼트', '구리', '하남', '안양', '시흥', '안산', '화성', '오산', '평택', '규모', '유세전', '강동', '중구', '금천', '의정부', '동두천', '파주', '고양', '김포', '부천', '이틀', '연속', '수도', '화력', '집중', '태세', '박빙', '구도', '최대', '표밭', '부동', '밀집', '수도', '민심', '대상', '선택', '집중', '전략', '풀이', '공식', '운동', '수도', '할애', '최대', '실정', '부각', '부동산', '일자리', '문제', '수도', '민심', '반응', '막판', '화력', '집중', '이유', '지사', '시장', '대장동', '개발', '특혜', '의혹', '주요', '공략', '안산시', '단원구', '안산', '문화', '광장', '국회', '기자단', '참여', '독려', '메시지', '경제', '발전', '청사진', '제시', '주력', '지지자', '참여', '당부', '주변', '연세', '거동', '불편', '주권', '행사', '장년', '참여', '심혈', '모습', '부동산', '정책', '실정', '비판', '집권', '공급', '집값', '마련', '지원', '목소리', '시흥', '고생', '이름', '가족', '아버지', '남자', '여자', '결혼', '부동산', '문제', '사회', '구리', '하남', '폭등', '전염', '전국', '고통', '이유', '정치', '지형', '유리', '주택', '정책', '주장', '바보멍충', '원색', '어휘', '비판', '대장', '의혹', '거론', '경제', '하남', '화성', '야권', '파트너', '합류'

In [6]:
len(data)

594

# Morphs 만들기

In [7]:
from kiwipiepy import Kiwi
import re
kiwi = Kiwi()
from kiwipiepy.utils import Stopwords
stopwords_dict = Stopwords()

# 사용자 정의 불용어를 키위 형태소 분석기의 불용어 사전에 추가하기
customized_stopwords = [
    '취재', '뉴스', '연합', '조선', '한겨레', '사전', '조사', '서울', '사진', '대표', '중앙',
    '유세', '더불어민주당', '국민의힘', '더불어 민주당', '국민의 힘', '허경영', '문재인', '박근혜',
    '원희룡', '윤석렬', '단일', '철수', '토론', '대선', '정권', '나라', '지지', '교체', '생각', '인간', '허경영'
    '정치', '기사제보', '연합뉴스', '재배포', '사진구매', '조선일보', '한겨레', '토트넘', '후보',
    '안철수', '윤석열', '이재명', '기자', '대통령', '광고', '이날', '오전', '오후', '관련', '이번',
    '시작', '당시', '가능', '이후', '이상', '공개', '자신', '당선', '시간', '시절', '이야기', '전날',
    '저녁', '지난해', '지금', '민주당', '국힘', '국민', '투표', '선거', '대한민국', '포함', '정도',
    '말씀', '해당', '일정', '일부', '하루', '발언', '지난달', '미래', '가운데', '경우', '부분',
    '정부', '국민의당', '의원', '포인트', '공동', '국민의 당', '경북', '제주', '호남', '영남', '대구', '대전',
    '경기', '한국', '부산', '충청', '광주', '전북', '전남', '충남', '성남', '만큼', '경남', '경기도', '심상정', '정의당'
]
for word in customized_stopwords:
    stopwords_dict.add((word, 'NNG'))
    stopwords_dict.add((word, 'NNP'))

In [8]:
def Kr_preprocessing(text):
    text = re.sub(r'[^\d\s\w]','',text)
    kiwi_tokens = kiwi.tokenize(text, stopwords=stopwords_dict)
    noun_words=[]
    for token in kiwi_tokens:
        if 'NN' in token.tag and len(token.form)>1:
            noun_words.append(token.form)
    return noun_words

In [9]:
# 윤석열
import os
import pickle

# 폴더 경로들 (각각의 폴더에 텍스트 파일이 있다고 가정)
folders = ['./윤석열 기사 200_연합뉴스/', './윤석열 기사 200_조선일보/', './윤석열 기사 200_한겨레/']  # 실제 폴더 경로로 수정
output_file = "ysy_morphs.p"  # 결과를 저장할 파일명

# 전처리 및 데이터 병합
data_dict = {}  # 결과를 저장할 딕셔너리
article_id = 1  # 고유 ID 시작값

for folder in folders:
    # 각 폴더의 모든 파일 읽기
    for file_name in os.listdir(folder):
        file_path = os.path.join(folder, file_name)
        if file_path.endswith(".txt"):  # 텍스트 파일만 처리
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
            
            # 전처리
            processed_content = Kr_preprocessing(content)
            
            # 딕셔너리 업데이트
            data_dict[article_id] = {
                "paper_id": str(article_id),
                "content": processed_content,
            }
            article_id += 1

# 결과 저장
with open(output_file, "wb") as f:
    pickle.dump(data_dict, f)

print(f"Processed data saved to {output_file}")

Processed data saved to ysy_morphs.p


In [10]:
# 이재명
import os
import pickle

# 폴더 경로들 (각각의 폴더에 텍스트 파일이 있다고 가정)
folders = ['./이재명 기사 200_연합뉴스/', './이재명 기사 200_조선일보/', './이재명 기사 200_한겨레/']  # 실제 폴더 경로로 수정
output_file = "ljm_morphs.p"  # 결과를 저장할 파일명

# 전처리 및 데이터 병합
data_dict = {}  # 결과를 저장할 딕셔너리
article_id = 1  # 고유 ID 시작값

for folder in folders:
    # 각 폴더의 모든 파일 읽기
    for file_name in os.listdir(folder):
        file_path = os.path.join(folder, file_name)
        if file_path.endswith(".txt"):  # 텍스트 파일만 처리
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
            
            # 전처리
            processed_content = Kr_preprocessing(content)
            
            # 딕셔너리 업데이트
            data_dict[article_id] = {
                "paper_id": str(article_id),
                "content": processed_content,
            }
            article_id += 1

# 결과 저장
with open(output_file, "wb") as f:
    pickle.dump(data_dict, f)

print(f"Processed data saved to {output_file}")

Processed data saved to ljm_morphs.p
